### 1. Collect the data

In [ ]:
import pandas as pd

linkWiki="https://en.wikipedia.org/wiki/Democracy_Index"
democracy=pd.read_html(linkWiki, header=0,attrs={"class":"wikitable sortable"})[4]

linkmil="https://www.cia.gov/the-world-factbook/field/military-expenditures/country-comparison"

milimoney=pd.read_html(linkmil)[0]

linkHDI="https://github.com/UW-eScience-WinterSchool/Python_Session/raw/main/countryCodesHDI.xlsx"
hdidata=pd.read_excel(linkHDI)

### 2. Check column names

In [ ]:
democracy.columns

In [ ]:
milimoney.columns

In [ ]:
hdidata.columns

You checked column names to name the key columns for merge, and to get rid of columns that may bring trouble.

In [ ]:
# this renaming will make merge easier
hdidata.rename(columns={'NAME':"Country"},inplace=True)

In [ ]:
# this dropping will avoid using RANK as a key.
democracy.drop(columns=["Rank"],inplace=True)
milimoney.drop(columns=["Rank"],inplace=True)

### 3. Merge

In [ ]:
allmerged=democracy.merge(milimoney)
allmerged=allmerged.merge(hdidata)
allmerged

### 4. Preprocessing

* _Check the strings in column names_:

In [ ]:
allmerged.columns.to_list()

* _Clean strings_:

In [ ]:
#replace '%' by "share"
allmerged.columns=allmerged.columns.str.replace("\%","share",regex=True)
#replace 'spaces' by "_"
allmerged.columns=allmerged.columns.str.replace("\s","_",regex=True)
#replace 'whatever is not a character' by ""
allmerged.columns=allmerged.columns.str.replace("\W","",regex=True)
#current names
allmerged.columns.to_list()

* _Drop MORE unneeded columns_:

In [ ]:
#take a look:
allmerged.columns[allmerged.columns.str.contains("Rank|Date|Δ",regex=True)]

In [ ]:
#save column to drop
toDrop=allmerged.columns[allmerged.columns.str.contains("Rank|Date|Δ",regex=True)]
# drop them
allmerged.drop(columns=toDrop,inplace=True)
# see result
allmerged

* _Look for missing values and check for wrong data types_:

In [ ]:
allmerged.info()

a. Some missing values can be corrected, others cannot:

In [ ]:
allmerged[allmerged.isnull().any(axis=1)]

We can not use Taiwan, but Namibia can be kept.

In [ ]:
allmerged.loc[:,'ISO2'][pd.isnull(allmerged.ISO2)]='NA'

Dropping rows with missing values:

In [ ]:
allmerged.dropna(inplace=True)
allmerged.reset_index(drop=True,inplace=True)
allmerged

b. Convert string to numerical

In [ ]:
toNumeric=['Overall_score',
 'Electoral_process_and_pluralism', 
 'Functioning_of_government',
 'Political_participation', 'Political_culture', 'Civil_liberties']
allmerged.loc[:,toNumeric]=allmerged.loc[:,toNumeric].apply(lambda x: pd.to_numeric(x))

c. Check ordinal variable:

In [ ]:
#check levels:
set(allmerged.Regime_type)

In [ ]:
levels=['Authoritarian', 'Hybrid regime','Flawed democracy', 'Full democracy']
allmerged.Regime_type=pd.Categorical(allmerged.Regime_type,
                                     categories=levels,
                                     ordered=True)

In [ ]:
allmerged.Regime_type

In [ ]:
# review:
allmerged.info()

### 4. Save your work

* _Save data for R_:

In [ ]:
from rpy2.robjects import pandas2ri
pandas2ri.activate()

from rpy2.robjects.packages import importr

base = importr('base')
base.saveRDS(allmerged,file="allmerged.rds")